In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

st.set_page_config(page_title="Strategic Profit Compass", layout="wide")

st.title("🧭 The Strategic Profit Compass")
st.markdown("### Bridging Operational Data to Executive Strategy")

# --- LOAD DATA ---
sales = pd.read_csv('sales_ledger.csv')
overheads = pd.read_csv('overheads.csv')

# --- SIDEBAR: CBDO STRATEGY SLIDERS ---
st.sidebar.header("Target Strategy (Year 3)")
price_change = st.sidebar.slider("Price Adjustment (%)", -20, 20, 5)
cost_reduction = st.sidebar.slider("Supplier Negotiation (Cost - %)", 0, 20, 10)

# --- CALCULATIONS ---
# Summarizing 2025 as our Baseline
current_year = sales[pd.to_datetime(sales['Date']).dt.year == 2025].copy()
total_rev = (current_year['Units_Sold'] * current_year['Price']).sum()
total_cogs = (current_year['Units_Sold'] * current_year['Unit_Cost']).sum()
total_fixed = overheads['Monthly_Cost_2025'].sum() * 12

# --- SECTION A: THE TRINITY KPIs ---
col1, col2, col3 = st.columns(3)
with col1:
    st.metric("Total Revenue", f"${total_rev:,.0f}")
with col2:
    margin = (total_rev - total_cogs) / total_rev
    st.metric("Gross Margin %", f"{margin:.1%}")
with col3:
    be_point = total_fixed / (current_year['Price'].mean() - current_year['Unit_Cost'].mean())
    st.metric("Break-Even Units", f"{be_point:,.0f}")

st.divider()

# --- SECTION B: THE SIMULATOR ---
st.subheader("🚀 Strategic Positioning Simulator")
st.write("How do your decisions today impact the bottom line tomorrow?")

# Simulation Logic
sim_price = current_year['Price'].mean() * (1 + price_change/100)
sim_cost = current_year['Unit_Cost'].mean() * (1 - cost_reduction/100)
sim_profit = (current_year['Units_Sold'].sum() * (sim_price - sim_cost)) - total_fixed

st.info(f"Projected Annual Operating Profit: **${sim_profit:,.2f}**")

# --- WATERFALL CALCULATION ENGINE ---
# Comparing 2024 (Baseline) to 2025 (Actual)
y1 = sales[pd.to_datetime(sales['Date']).dt.year == 2024]
y2 = sales[pd.to_datetime(sales['Date']).dt.year == 2025]

# Financial Components
y1_profit = (y1['Units_Sold'] * (y1['Price'] - y1['Unit_Cost'])).sum() - (overheads['Monthly_Cost_2024'].sum() * 12)
y2_profit = (y2['Units_Sold'] * (y2['Price'] - y2['Unit_Cost'])).sum() - (overheads['Monthly_Cost_2025'].sum() * 12)

# Differences
volume_impact = (y2['Units_Sold'].sum() - y1['Units_Sold'].sum()) * (y1['Price'].mean() - y1['Unit_Cost'].mean())
price_impact = y2['Units_Sold'].sum() * (y2['Price'].mean() - y1['Price'].mean())
cost_impact = y2['Units_Sold'].sum() * (y1['Unit_Cost'].mean() - y2['Unit_Cost'].mean()) # Negative if costs went up
fixed_cost_impact = (overheads['Monthly_Cost_2024'].sum() * 12) - (overheads['Monthly_Cost_2025'].sum() * 12)

# --- THE WATERFALL CHART ---
fig = go.Figure(go.Waterfall(
    name = "Profit Bridge", orientation = "v",
    measure = ["relative", "relative", "relative", "relative", "total"],
    x = ["2024 Profit", "Volume Growth", "Price Increase", "Cost Changes", "2025 Profit"],
    textposition = "outside",
    text = [f"${y1_profit:,.0f}", f"{volume_impact:,.0f}", f"{price_impact:,.0f}", f"{cost_impact:,.0f}", f"Total: ${y2_profit:,.0f}"],
    y = [y1_profit, volume_impact, price_impact, cost_impact, y2_profit],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
    increasing = {"marker":{"color":"#2ECC71"}},
    decreasing = {"marker":{"color":"#E74C3C"}},
    totals = {"marker":{"color":"#3498DB"}}
))

fig.update_layout(title = "Annual Profit Bridge (Year-over-Year)", showlegend = False)
st.plotly_chart(fig, use_container_width=True)

# --- FINANCIAL STATEMENTS ---
st.divider()
st.subheader("🏛️ The Financial Trinity (Managerial Ratios)")
c1, c2, c3 = st.columns(3)

with c1:
    st.write("**Balance Sheet Health**")
    st.info("Current Ratio: 1.8") # You can calculate these from a dummy BS CSV
    st.caption("Target: > 1.5 (Strong Liquidity)")

with c2:
    st.write("**Cash Flow Vitality**")
    st.success("Operating Cash Flow: $142k")
    st.caption("Cash generated from core ops.")

with c3:
    st.write("**Strategic Positioning**")
    st.warning("CAC/LTV Ratio: 1:2.8")
    st.caption("Target: 1:3.0 (Scale-up Ready)")